In [1]:
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import RMSprop, SGD
import pandas as pd
import numpy as np

/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Split file by ticker
filename = 'goog_avg.csv'
full = pd.read_csv(filename)
full = full.drop(labels=['Unnamed: 0'], axis=1)
top_20 = ['FB','AAPL','GOOG','INTU','ALGN','AMZN','AAL','MSFT','AMAT','FOX',
          'SBUX','WDC','NFLX','COST','ADBE','EBAY','WDAY','LRCX','BIDU','PYPL']
filelist = []
for elem in top_20:
    tmp = full.loc[full['Ticker']==elem]
    savename = 'ForRNN/' + elem +'.csv'
    filelist.append(savename)
    tmp.to_csv(savename)
    print(savename)

ForRNN/FB.csv
ForRNN/AAPL.csv
ForRNN/GOOG.csv
ForRNN/INTU.csv
ForRNN/ALGN.csv
ForRNN/AMZN.csv
ForRNN/AAL.csv
ForRNN/MSFT.csv
ForRNN/AMAT.csv
ForRNN/FOX.csv
ForRNN/SBUX.csv
ForRNN/WDC.csv
ForRNN/NFLX.csv
ForRNN/COST.csv
ForRNN/ADBE.csv
ForRNN/EBAY.csv
ForRNN/WDAY.csv
ForRNN/LRCX.csv
ForRNN/BIDU.csv
ForRNN/PYPL.csv


In [3]:
# Select one ticker
ticker = pd.read_csv('ForRNN/FB.csv')
ticker = ticker.drop(labels=['Unnamed: 0','Ticker','Sector'], axis=1)

In [7]:
# Train and store the best performing model
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  
from sklearn import preprocessing
full = pd.read_csv('goog_avg.csv')
full = full.drop(labels=['Unnamed: 0'], axis=1)
full['Ticker'] = pd.Categorical(full['Ticker'])
full['Sector'] = pd.Categorical(full['Sector'])
#for libor, just use yesterday's if no value
full['Libor'].fillna(method='pad', inplace=True)
#for news, just use yesterday's if no value
for i in range(0, 40):
    full[str(i)].fillna(method='pad', inplace=True)
#drop the rows with no value change (first 30 days)
full = full.dropna(subset=['Thirty-day Change (%)'])
full = full.drop(labels=['High','Low', 'Open', 'Close', 'Adj Close'], axis=1)
categorical = ['Ticker','Sector']
file_one_hot_encoded = pd.get_dummies(full, columns=categorical, drop_first=True)
in_set = file_one_hot_encoded
in_set.Date = pd.to_datetime(in_set.Date)
in_set['y-label'] = pd.Categorical(in_set['y-label'])
in_train = in_set[(in_set['Date'] <'2017-01-01')]
in_test = in_set[(in_set['Date'] >='2017-01-01')]
y_train = in_train['y-label']
y_test = in_test['y-label']
X_train = in_train.drop(labels=['Date', 'y-label'], axis=1)
X_test = in_test.drop(labels=['Date', 'y-label'], axis=1)
X_train['Volume'] = np.log(X_train['Volume']+0.5)
X_test['Volume'] = np.log(X_test['Volume']+0.5)
# c = 0.5
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
svm_rbf = svm.SVC(C=0.5, kernel='rbf',max_iter=50000)
svm_rbf.fit(X_train, y_train)
y_hat_train = svm_rbf.predict(X_train)
print(y_hat_train)
y_hat_test = svm_rbf.predict(X_test)
print(y_hat_test)

In [11]:
# Load data for plot
full_2 = pd.read_csv('goog_avg.csv')
full_2 = full_2.drop(labels=['Unnamed: 0'], axis=1)
full_2.Date = pd.to_datetime(full.Date)
full_test = full_2[(full_2['Date'] >='2017-01-01')]
full_test = full_test.reset_index(drop = True)
y_hat_test = pd.DataFrame(y_hat_test).reset_index(drop = True)
test = full_test.join(y_hat_test)
dif = np.zeros(full_test.shape[0])
dif[test['y-label'] != test[0]] = 1
to_plot = full_test[['Date','Ticker','Open','High','Low','Close']]
dif = pd.DataFrame(dif).reset_index(drop = True)
to_plot = to_plot.join(dif)
FB = to_plot.loc[to_plot['Ticker'] == 'FB']
correct = FB.loc[FB[0] == 0]
wrong = FB.loc[FB[0] == 1]
wrong2 = wrong.rename(columns={"Open": "new_Close", "Close": "new_Open","High": "new_Low", "Low": "new_High"})
wrong2 = wrong2.rename(columns={"new_Close": "Close", "new_Open": "Open","new_Low": "Low", "new_High": "High"})
ready_to_plot = pd.concat([correct, wrong2], ignore_index = True)
ready_to_plot.Date = pd.to_datetime(ready_to_plot.Date)
ready_to_plot = ready_to_plot.sort_values(by=['Date'])

In [71]:
ticker = ['FB_pred']*ready_to_plot.shape[0]
ticker = pd.DataFrame(ticker).reset_index(drop = True)
ready_to_plot = pd.DataFrame(ready_to_plot).reset_index(drop = True)
ticker = ticker.rename(columns={0: "new_ticker"})
new_plot = ready_to_plot.join(ticker)
new_plot = new_plot.drop(labels=['Ticker'], axis=1)
new_plot = new_plot.rename(columns={"new_ticker":"Ticker"})
true_FB = to_plot.loc[to_plot['Ticker'] == 'FB']
true_FB['High'] = true_FB['High']+20
true_FB['Low'] = true_FB['Low']+20
true_FB['Close'] = true_FB['Close']+20
true_FB['Open'] = true_FB['Open']+20
new_plot = pd.concat([new_plot, true_FB], ignore_index = True)

In [79]:
# import plotly
import plotly
plotly.tools.set_credentials_file(username='get a username', api_key='get an API')
import plotly.plotly as py
# plotly.tools.set_credentials_file(username='ylxu', api_key='HBPTupzkIc99pbjp0TS9')
import plotly.graph_objs as go
import pandas_datareader as web
from datetime import datetime

# df = web.DataReader("aapl", 'morningstar').reset_index()

trace = go.Candlestick(x=new_plot.Date,
                       open=new_plot.Open,
                       high=new_plot.High,
                       low=new_plot.Low,
                       close=new_plot.Close)
data = [trace]
py.iplot(data, filename='simple_candlestick')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~ylxu/0 or inside your plot.ly account where it is named 'simple_candlestick'
